In [1]:
import numpy as np
from scipy.optimize import Bounds, NonlinearConstraint
import sklearn.gaussian_process as gpr
import time
import RXTR_SYSTEM
from joblib import Parallel, delayed

import sys
sys.path.append('./../../BO_algos')
import Parallel_Algos as BO_algos

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter('ignore', RuntimeWarning)
warnings.simplefilter('ignore', ConvergenceWarning)

In [83]:
# Define Parameters
exp_w = 2.6
C0varf = np.loadtxt('C_var_gp.txt')
FR = np.array([0.100, 0.075, 0.075])
R_Frac = 1e-6
ub = np.array([423, 423])
lb = np.array([303, 303])
bounds = Bounds((0, 0), (1, 1))
dim = len(ub)
kernel = gpr.kernels.Matern((1, 1), ((0.06, 5), (0.06, 5)), nu = 2.5)

In [85]:
# Set system and reference functions
SYST_RECYCLE = RXTR_SYSTEM.SYST_RECYCLE
SYST_RECYCLE_REF = RXTR_SYSTEM.RXTR_REG()

Calculation of reaction model weights...
2.9305176734924316


In [43]:
# Train reference model (GP)
def scale(x, ub = 423, lb = 303, sf = 1):
    m = sf/(ub - lb)
    b = -lb*m
    return m*x+b

In [44]:
TT = np.arange(303, 424, 1)
TT = np.meshgrid(TT, TT)
TT = np.hstack([TT[0].reshape(-1, 1), TT[1].reshape(-1, 1)])
TT.shape

(14641, 2)

In [69]:
print('Create Statistical Reference Model...')
start = time.time()

Tmod = np.linspace(303, 423, 13)
Tmod = np.meshgrid(Tmod, Tmod)
Tmod = np.hstack([Tmod[0].reshape(-1, 1), Tmod[1].reshape(-1, 1)])
CTREF = np.ones((Tmod.shape[0], 3))

Ctref = Parallel(n_jobs = 5)(delayed(SYST_RECYCLE_REF)(Tmod, FR, R_Frac, Cdist)
                             for Cdist in C0varf)

for i in range(3):
    C = np.vstack(Ctref[:][:]).T[:, i::3]
    C = 8*np.sum(C, axis = 1)
    CTREF[:, i] = C
    kergp = gpr.kernels.Matern((5, 5), ((1, 10), (1, 10)), nu = 2.5)
gprefmod = gpr.GaussianProcessRegressor(kernel = kergp,
                                        alpha = 1e-6,
                                        n_restarts_optimizer = 10,
                                        normalize_y = True)
gprefmod.fit(scale(Tmod), CTREF)

end = time.time()
mobdtm = end-start
print(mobdtm)

Create Statistical Reference Model...


KeyboardInterrupt: 

In [ ]:
Tmod

In [36]:
def SYST_C(T):
    CtR = 0
    for i in range(C0varf.shape[0]):
        Ctr = 8*SYST_RECYCLE(T, FR, R_Frac, C0varf[i])[-1]
        CtR += Ctr
    return CtR

def SYST_C_DIST(T):
    CtR = 0
    for i in range(C0varf.shape[0]):
        Ctr = SYST_RECYCLE(T, FR, R_Frac, C0varf[i])
        Ctr = np.vstack(Ctr[:]).T
        CtR += 8*Ctr
    return CtR

def SYST_C_REFGP(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)[:, -1]

def SYST_C_DISTGP(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)

def SYST_C_REF1(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)[:, 0]

def SYST_C_REF2(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)[:, 1]

def zr(x):
    x = x.reshape(-1, 1)
    x = x.reshape(int(x.shape[0]/2), 2)
    return np.zeros((x.shape[0], 3))


In [37]:
# Setup BO class
REACOPTIM = BO_algos.BO(ub = ub,
                        lb = lb,
                        dim = 2,
                        exp_w = exp_w,
                        kernel = kernel,
                        system = SYST_C,
                        bounds = bounds,
                        **{'refmod': SYST_C_REFGP,
                           'distmod': SYST_C_DIST,
                           'ref_distmod': SYST_C_DISTGP,
                           'ref_distmod1': SYST_C_REF1,
                           'ref_distmod2': SYST_C_REF2})


In [11]:
## Generate level sets
gpparts = gpr.GaussianProcessRegressor(kernel = kergp,
                                       alpha = 1e-6,
                                       n_restarts_optimizer = 10,
                                       normalize_y = True)
gpparts.fit(scale(Tmod), (CTREF[:, -1]).reshape(-1, 1))

parts = np.array([-461, -383]) # ref mod with g_1(T_1) and g_2(T_1, T_2)

con1 = lambda x: (gpparts.predict(x.reshape(1, 2))).flatten()
con2 = lambda x: x[0]

nlc1 = NonlinearConstraint(con1, -1e4, parts[0])
nlc21 = NonlinearConstraint(con1, parts[0], parts[1])
nlc22 = NonlinearConstraint(con2, 0.4, 1.1)
nlc31 = NonlinearConstraint(con1, parts[0], parts[1])
nlc32 = NonlinearConstraint(con2, 0, 0.38)
nlc4 = NonlinearConstraint(con1, parts[1], 1e4)

cons = {'1': [nlc1], '2': [nlc21, nlc22], '3': [nlc31, nlc32], '4': [nlc4]}

NameError: name 'kergp' is not defined

In [13]:
## VP-BO intial points
lim_init = REACOPTIM.scale(np.array([336, 380]))


# Run_BO
trials_seq = 100
trials_par1 = 25
trials_par2 = 33

x_init = np.linspace(bounds.lb, bounds.ub, 5)
x_init = np.meshgrid(*x_init.T)
x_init = np.reshape(x_init, (dim, -1)).T


In [14]:
MET_SBO = np.zeros((trials_seq, 4))
PARAMS_SBO = np.ones(x_init.shape)
DIST_SBO = np.array([]).reshape(0, dim)
RES_SBO = np.ones((MET_SBO.shape[0], x_init.shape[0]))

MET_REFBO = np.zeros((trials_seq, 4))
PARAMS_REFBO = np.ones(x_init.shape)
DIST_REFBO = np.array([]).reshape(0, dim)
RES_REFBO = np.ones((MET_REFBO.shape[0], x_init.shape[0]))

MET_LSBO1 = np.zeros((trials_par1, 4))
PARAMS_LSBO1 = np.ones(x_init.shape)
DIST_LSBO1 = np.array([]).reshape(0, dim)
RES_LSBO1 = np.ones((MET_LSBO1.shape[0], x_init.shape[0]))

MET_LSBO2 = np.zeros((trials_par1, 4))
PARAMS_LSBO2 = np.ones(x_init.shape)
DIST_LSBO2 = np.array([]).reshape(0, dim)
RES_LSBO2 = np.ones((MET_LSBO2.shape[0], x_init.shape[0]))

MET_VPBO1 = np.zeros((trials_par2, 4))
PARAMS_VPBO1 = np.ones(x_init.shape)
DIST_VPBO1 = np.array([]).reshape(0, dim)
RES_VPBO1 = np.ones((MET_VPBO1.shape[0], x_init.shape[0]))

MET_VPBO2 = np.zeros((trials_par2, 4))
PARAMS_VPBO2 = np.ones(x_init.shape)
DIST_VPBO2 = np.array([]).reshape(0, dim)
RES_VPBO2 = np.ones((MET_VPBO2.shape[0], x_init.shape[0]))

MET_HSBO = np.zeros((trials_par1, 4))
PARAMS_HSBO = np.ones(x_init.shape)
DIST_HSBO = np.array([]).reshape(0, dim)
RES_HSBO = np.ones((MET_HSBO.shape[0], x_init.shape[0]))

MET_EXBO = np.zeros((trials_par1, 4))
PARAMS_EXBO = np.ones(x_init.shape)
DIST_EXBO = np.array([]).reshape(0, dim)
RES_EXBO = np.ones((MET_EXBO.shape[0], x_init.shape[0]))

MET_NMCBO = np.zeros((trials_par1, 4))
PARAMS_NMCBO = np.ones(x_init.shape)
DIST_NMCBO = np.array([]).reshape(0, dim)
RES_NMCBO = np.ones((MET_NMCBO.shape[0], x_init.shape[0]))

MET_QBO = np.zeros((trials_par1, 4))
PARAMS_QBO = np.ones(x_init.shape)
DIST_QBO = np.array([]).reshape(0, dim)
RES_QBO = np.ones((MET_QBO.shape[0], x_init.shape[0]))


## S-BO
for i, x_0 in enumerate(x_init):  
    start = time.time()
    REACOPTIM.optimizer_sbo(trials = trials_seq, x_init = x_0)
    end = time.time()
    print('Run time '+str(end-start)+'s')
    print('iteration '+str(i+1))
    MET_SBO[:, 0] += REACOPTIM.time_sbo.flatten()
    MET_SBO[:, 1] += REACOPTIM.time_fsbo.flatten()
    MET_SBO[:, 2] += MET_SBO[:, 2]+REACOPTIM.y_sbo.flatten()
    MET_SBO[:, 3] += MET_SBO[:, 3]+np.min(REACOPTIM.y_sbo)
    print('Best S-BO value is '+str(np.min(REACOPTIM.y_sbo)))
    PARAMS_SBO[i] = REACOPTIM.x_sbo[np.argmin(REACOPTIM.y_sbo)]
    DIST_SBO = np.vstack([DIST_SBO, REACOPTIM.x_sbo])
    RES_SBO[:, i] = REACOPTIM.y_sbo[:, 0]
MET_SBO[:, 0] = MET_SBO[:, 0]/(i+1)
MET_SBO[:, 1] = MET_SBO[:, 1]/(i+1)
MET_SBO[:, 2] = MET_SBO[:, 2]/(i+1)
MET_SBO[:, 3] = MET_SBO[:, 3]/(i+1)
REACOPTIM.y_sbo = MET_SBO[:, 2].flatten()
REACOPTIM.time_sbo = MET_SBO[:, 0].flatten()
REACOPTIM.time_fsbo = MET_SBO[:, 1].flatten()
    

## Ref-BO
for i, x_0 in enumerate(x_init):  
    start = time.time()
    REACOPTIM.optimizer_refbo(trials = trials_seq, x_init = x_0)
    end = time.time()
    print('Run time '+str(end-start)+'s')
    print('iteration '+str(i+1))
    MET_REFBO[:, 0] += REACOPTIM.time_ref.flatten()
    MET_REFBO[:, 1] += REACOPTIM.time_fref.flatten()
    MET_REFBO[:, 2] += REACOPTIM.y_ref.flatten()
    MET_REFBO[:, 3] += np.min(REACOPTIM.y_ref)
    print('Best Ref-BO value is '+str(np.min(REACOPTIM.y_ref)))
    PARAMS_REFBO[i] = REACOPTIM.x_ref[np.argmin(REACOPTIM.y_ref)]
    DIST_REFBO = np.vstack([DIST_REFBO, REACOPTIM.x_ref])
    RES_REFBO[:, i] = REACOPTIM.y_ref[:, 0]
MET_REFBO[:, 0] = MET_REFBO[:, 0]/(i+1)
MET_REFBO[:, 1] = MET_REFBO[:, 1]/(i+1)
MET_REFBO[:, 2] = MET_REFBO[:, 2]/(i+1)
MET_REFBO[:, 3] = MET_REFBO[:, 3]/(i+1)
REACOPTIM.y_ref = MET_REFBO[:, 2].flatten()
REACOPTIM.time_ref = MET_REFBO[:, 0].flatten()
REACOPTIM.time_fref = MET_REFBO[:, 1].flatten()


Vanilla BO run...


ValueError: cannot reshape array of size 2 into shape (3)

In [ ]:
## LS-BO
for i, x_0 in enumerate(x_init):
    start = time.time()
    REACOPTIM.optimizer_lsbo(trials = trials_par1,
                             partition_number = 4,
                             repartition_intervals = [],
                             x_samps = [],
                             f_cores = 4,
                             af_cores = 1,
                             ref_cores = 1,
                             partitions = cons, 
                             x_init = x_0)
    end = time.time()
    print('Run time '+str(end-start)+'s')
    print('iteration '+str(i+1))
    MET_LSBO1[:, 0] += REACOPTIM.time_ls.flatten()
    MET_LSBO1[:, 1] += REACOPTIM.time_fls.flatten()
    MET_LSBO1[:, 2] += REACOPTIM.y_lsbst.flatten()
    MET_LSBO1[:, 3] += np.min(REACOPTIM.y_lsbst)
    print('Best LS-BO value is '+str(np.min(REACOPTIM.y_lsbst)))
    PARAMS_LSBO1[i] = REACOPTIM.x_ls[np.argmin(REACOPTIM.y_ls)]
    DIST_LSBO1 = np.vstack([DIST_LSBO1, REACOPTIM.x_ls])
    RES_LSBO1[:, i] = REACOPTIM.y_lsbst[:, 0]
MET_LSBO1[:, 0] = MET_LSBO1[:, 0]/(i+1)
MET_LSBO1[:, 1] = MET_LSBO1[:, 1]/(i+1)
MET_LSBO1[:, 2] = MET_LSBO1[:, 2]/(i+1)
MET_LSBO1[:, 3] = MET_LSBO1[:, 3]/(i+1)
REACOPTIM.y_lsbst = MET_LSBO1[:, 2].flatten()
REACOPTIM.time_ls = MET_LSBO1[:, 0].flatten()
REACOPTIM.time_fls = MET_LSBO1[:, 1].flatten()


In [68]:
## VP-BO 
for i, x_0 in enumerate(x_init):
    start = time.time()
    REACOPTIM.optimizer_vpbo(trials = trials_par2,
                             split_num = 3,
                             lim_init = lim_init,
                             f_cores = 3,
                             af_cores = 1,
                             ref_cores = 1,
                             x_init  = x_0)
    end = time.time()
    print('Run time '+str(end-start)+'s')
    print('iteration '+str(i+1))
    MET_VPBO1[:, 0] += REACOPTIM.time_vp.flatten()
    MET_VPBO1[:, 1] += REACOPTIM.time_fvp.flatten()
    MET_VPBO1[:, 2] += REACOPTIM.y_vpbst[:, -1].flatten()
    MET_VPBO1[:, 3] += np.min(REACOPTIM.y_vpbst[:, -1])
    print('Best VP-BO value is '+str(np.min(REACOPTIM.y_vpbst[:, -1])))
    PARAMS_VPBO1[i] = REACOPTIM.x_vp[np.argmin(REACOPTIM.y_vp[:, -1])]
    DIST_VPBO1 = np.vstack([DIST_VPBO1, REACOPTIM.x_vp])
    RES_VPBO1[:, i] = REACOPTIM.y_vpbst[:, -1]
MET_VPBO1[:, 0] = MET_VPBO1[:, 0]/(i+1)
MET_VPBO1[:, 1] = MET_VPBO1[:, 1]/(i+1)
MET_VPBO1[:, 2] = MET_VPBO1[:, 2]/(i+1)
MET_VPBO1[:, 3] = MET_VPBO1[:, 3]/(i+1)
REACOPTIM.y_vpbst = MET_VPBO1[:, 2].reshape(-1, 1)
REACOPTIM.time_vp = MET_VPBO1[:, 0].flatten()
REACOPTIM.time_fvp = MET_VPBO1[:, 1].flatten()

NameError: name 'x_init' is not defined

In [30]:
# Plot convergence plots
REACOPTIM.plots('R1')
REACOPTIM.plots_time('R1')
REACOPTIM.plot_exptime('R1')

NameError: name 'REACOPTIM' is not defined

----------------------------------------------------------------------------------------------

In [7]:
import numpy as np
from scipy.optimize import Bounds, NonlinearConstraint
import sklearn.gaussian_process as gpr
import time
import RXTR_SYSTEM
from joblib import Parallel, delayed

import sys
sys.path.append('./../../BO_algos')
import Parallel_Algos as BO_algos
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter('ignore', RuntimeWarning)
warnings.simplefilter('ignore', ConvergenceWarning)

In [8]:
import pandas as pd
import torch

# Load the data
df = pd.read_csv('df_scaled.csv')
df = df.drop_duplicates().dropna()

In [9]:
df.head(-1)

,surfaceProcessingLocationCavity,hotRunner,hotRunnerCost,manufacturingCost,mouldValidationCost,designTime,hourlyDesignCost,removedChassis,weightChassisProcessed,surfaceProcessingLocationChassis,...,human health - ionising radiation,human health - respiratory effects (inorganics),human health - total,human health - human toxicity,ecosystem quality - aquatic ecotoxicity,climate change - climate change,human health - ozone layer depletion,ecosystem quality - land occupation,climate change - total,ecosystem quality - total
0,0.5,1.0,1.0,0.5,0.0,0.0,1.0,0.666667,0.0,0.000000,...,0.000524,0.001147,0.001123,0.003098,0.001833,0.000650,0.000886,0.002312,0.000650,0.003202
1,0.5,1.0,1.0,0.0,0.5,0.5,0.0,0.000000,0.0,0.000000,...,0.000075,0.000045,0.000054,0.000039,0.000029,0.000032,0.000043,0.000055,0.000032,0.000038
2,0.5,0.0,1.0,0.5,1.0,1.0,1.0,0.666667,0.0,0.000000,...,0.000522,0.001143,0.001119,0.003088,0.001827,0.000647,0.000883,0.002304,0.000647,0.003192
3,0.5,1.0,0.0,0.0,1.0,0.5,0.0,0.400000,0.0,0.000000,...,0.000315,0.000690,0.000675,0.001863,0.001102,0.000391,0.000533,0.001390,0.000391,0.001926
4,0.5,0.0,0.5,0.0,0.5,0.0,1.0,0.400000,0.0,0.000000,...,0.000313,0.000686,0.000671,0.001853,0.001096,0.000388,0.000530,0.001382,0.000388,0.001915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164142,1.0,0.0,1.0,1.0,0.0,0.5,0.0,0.400000,0.0,0.666667,...,0.002785,0.003425,0.004098,0.012891,0.029770,0.001492,0.001615,0.002877,0.001492,0.058132
164143,0.5,1.0,0.0,0.0,0.0,1.0,0.0,0.400000,0.0,0.000000,...,0.010388,0.006251,0.007575,0.005804,0.004219,0.004516,0.005981,0.007872,0.004516,0.005611
164144,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.017868,0.009871,0.012244,0.007008,0.005538,0.007321,0.009669,0.011507,0.007321,0.006554
164145,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.000000,0.0,1.000000,...,0.010242,0.008582,0.010265,0.020744,0.040248,0.004851,0.006126,0.009296,0.004851,0.077022


624


2496

In [10]:
label_col=        ['human health - total']

In [11]:
feature_cols= ['manufacturingCost','nProdottiAnno', 'hotRunner']#human health-total

In [12]:
df = df[feature_cols + label_col]
df = df.drop_duplicates().dropna()

In [13]:
n_test= round(0.2*  df.shape[0])
print(n_test)
n_train= df.shape[0]-n_test
n_train

1234


4938

In [14]:
# Assume last column is the target, rest are features
X = df.iloc[:, :-1].values  # Features
y = df.iloc[:, :1].values   # Target
X.shape

(6172, 3)

In [15]:
  # or whatever number you want for n_test
train_x = torch.tensor(X[:n_train], dtype=torch.float32)
train_y = torch.tensor(y[:n_train], dtype=torch.float32)

In [16]:
test_x = train_x[:-n_test]  # Select first 1000 as test data
test_y = train_y[:-n_test]

In [17]:
print(train_x.size())
print(test_x.size())

torch.Size([4938, 3])
torch.Size([3704, 3])


In [18]:
train_x.max()

tensor(1.)

In [26]:
# Train reference model (GP)
def scale(x, ub = 1, lb = 0, sf = 1):
    m = sf/(ub - lb)
    b = -lb*m
    return m*x+b

In [82]:
C0varf

NameError: name 'C0varf' is not defined

In [19]:
TT = train_x


In [20]:
TT.shape

torch.Size([4938, 3])

In [65]:
print('Create Statistical Reference Model...')
start = time.time()
Tmod= TT

CTREF= train_y


for i in range(1):
    kergp = gpr.kernels.Matern(5, (1, 10), nu = 2.5)
gprefmod = gpr.GaussianProcessRegressor(kernel = kergp,
                                        alpha = 1e-6,
                                        n_restarts_optimizer = 1,
                                        normalize_y = True)
gprefmod.fit(Tmod, CTREF)

end = time.time()
mobdtm = end-start
print(mobdtm)

Create Statistical Reference Model...
82.20933318138123


In [23]:
# Define Parameters
exp_w = 2.6
#C0varf = np.loadtxt('C_var_gp.txt')
FR = np.array([0.100, 0.075, 0.075])
R_Frac = 1e-6
ub = np.array([1,1,1])
lb = np.array([0,0,0])
bounds = Bounds(np.zeros(len(ub)), np.ones(len(ub)))
dim = len(ub)
kernel = gpr.kernels.Matern((1, 1), ((0.06, 5), (0.06, 5)), nu = 2.5)

In [55]:
dim

3

In [24]:
bounds

Bounds(array([0., 0., 0.]), array([1., 1., 1.]))

In [56]:
REACOPTIM = BO_algos.BO(ub = ub,
                        lb = lb,
                        dim = dim,
                        exp_w = exp_w,
                        kernel = kernel,
                        system = 0,
                        bounds = bounds,
                        )


In [66]:
REACOPTIM = BO_algos.BO(ub = ub,
                        lb = lb,
                        dim = dim,
                        exp_w = exp_w,
                        kernel = kernel,
                        system = SYST_C,
                        bounds = bounds,
                        **{'refmod': SYST_C_REFGP,
                           'distmod': SYST_C_DIST,
                           'ref_distmod': SYST_C_DISTGP,
                           'ref_distmod1': SYST_C_REF1,
                           'ref_distmod2': SYST_C_REF2})


In [67]:
scaled_output = REACOPTIM.scale(np.zeros(train_x.shape))
print("Shape of scaled_output:", scaled_output.shape)


Shape of scaled_output: (4938, 3)


In [89]:
## VP-BO intial points
lim_init = REACOPTIM.scale(np.zeros(train_x[1].shape))


# Run_BO
trials_seq = 1000
trials_par1 = 3
trials_par2 = 3

x_init = np.linspace(bounds.lb, bounds.ub, 5)
x_init = np.meshgrid(*x_init.T)
x_init = np.reshape(x_init, (dim, -1)).T

In [90]:
x_init = test_x

In [91]:
x_init.shape

torch.Size([3704, 3])

In [71]:
MET_VPBO1 = np.zeros((trials_par2, 4))
PARAMS_VPBO1 = np.ones(x_init.shape)
DIST_VPBO1 = np.array([]).reshape(0, dim)
RES_VPBO1 = np.ones((MET_VPBO1.shape[0], x_init.shape[0]))


In [92]:
## VP-BO 
for i, x_0 in enumerate(x_init):
    start = time.time()
    REACOPTIM.optimizer_vpbo(trials = trials_par2,
                             split_num = 3,
                             lim_init = lim_init,
                             f_cores =3,
                             af_cores = 1,
                             ref_cores = 1,
                             x_init  = x_0)
    end = time.time()
    print('Run time '+str(end-start)+'s')
    print('iteration '+str(i+1))
    MET_VPBO1[:, 0] += REACOPTIM.time_vp.flatten()
    MET_VPBO1[:, 1] += REACOPTIM.time_fvp.flatten()
    MET_VPBO1[:, 2] += REACOPTIM.y_vpbst[:, -1].flatten()
    MET_VPBO1[:, 3] += np.min(REACOPTIM.y_vpbst[:, -1])
    print('Best VP-BO value is '+str(np.min(REACOPTIM.y_vpbst[:, -1])))
    PARAMS_VPBO1[i] = REACOPTIM.x_vp[np.argmin(REACOPTIM.y_vp[:, -1])]
    DIST_VPBO1 = np.vstack([DIST_VPBO1, REACOPTIM.x_vp])
    RES_VPBO1[:, i] = REACOPTIM.y_vpbst[:, -1]
MET_VPBO1[:, 0] = MET_VPBO1[:, 0]/(i+1)
MET_VPBO1[:, 1] = MET_VPBO1[:, 1]/(i+1)
MET_VPBO1[:, 2] = MET_VPBO1[:, 2]/(i+1)
MET_VPBO1[:, 3] = MET_VPBO1[:, 3]/(i+1)
REACOPTIM.y_vpbst = MET_VPBO1[:, 2].reshape(-1, 1)
REACOPTIM.time_vp = MET_VPBO1[:, 0].flatten()
REACOPTIM.time_fvp = MET_VPBO1[:, 1].flatten()

Variable Partitioned BO run...


ValueError: cannot reshape array of size 3 into shape (1,2)

In [78]:
lim_init

array([0., 0., 0.])

In [79]:
np.ones((3, 3))

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [81]:
x = lim_init*np.ones((3, 3))
x

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])